## Create copy of this colab and code

# Practice Questions for Day2

Download the data from: https://drive.google.com/drive/folders/1bseMSE_CMSvxHqH2O-utb-pqailyo3lQ?usp=sharing **(Download MLP Boot Camp dataset Day2)**
Upload it on your google drive(Upload it to a folder) and mount your google drive. Once it's done, click on `Files` option on the left sidebar of your colab notebook. You will be able to locate your Google Drive(My Drive) & locate the train.csv file. Copy the path.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/MLP BootCamp/Copy of MLP Boot Camp dataset Day2.csv")
df.head()

,Gender,Age,HasTension,AnyHeartDisease,NeverMarried,Occupation,LivesIn,GlucoseLevel,BMI,SmokingStatus,HeartAttack
0,Male,63.00,No,No,Yes,Self-employed,Village,78.5,18.6,never smoked,No
1,Male,57.00,No,No,Yes,Private,City,84.2,31.2,Unknown,Yes
2,Male,81.00,No,No,Yes,Self-employed,City,59.9,25.5,smokes,No
3,Female,1.24,No,No,No,children,Village,113.3,20.3,Unknown,No
4,Male,16.00,No,No,No,Never_worked,City,60.9,27.8,never smoked,No


In [ ]:
df["SmokingStatus"].value_counts()

SmokingStatus
never smoked       1485
Unknown            1214
formerly smoked     691
smokes              610
Name: count, dtype: int64

## Q1) Divide the data into training and test sets
<!-- - Replace the respective missing or unknown values in features room count, riverside and age with `np.nan`. -->
- Keep 30% of the data as test set.
- Use random_state as 0
- `HeartAttack` is the target, rest of the columns are the features.
- For the label/target vector, replace "Yes" with 1 and "No" with 0.

- Divide the dataset into training and test sets keeping target(y) in **stratified** manner.


**Hint**: look for the documentation of the usual function that divides the data into training and test datasets.

In [ ]:
X = df.drop("HeartAttack", axis=1)
y = df["HeartAttack"].copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(y)
y

array([0, 1, 0, ..., 0, 0, 0])

In [ ]:
X.shape

(4000, 10)

In [ ]:
le.inverse_transform([1])

array(['Yes'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0, stratify=y)

##Q2) Prepare a data preprocessing pipeline to process features in following order:

1. `Gender`: Impute with most frequent then ordinally encode.
2. `Age`: Impute with mean then standard scale.
3. `HasTension`: Ordinally encode.
4. `AnyHeartDisease`:Ordinally encode.
5. `NeverMarried`:Ordinally encode.
6. `Occupation`: One hot encode.
7. `LivesIn`: Impute with most frequent then ordinally encode.
8. `GlucoseLevel`: Impute with mean, then min-max scaling.
9. `BMI`: Impute with mean, then standard scale.
10. `SmokingStatus`: Impute with most frequent, then one hot encode.

Hint: After transformation, your feature matrix must have columns in following order:

0. Gender
1. Age
2. HasTension
3. AnyHeartDisease
4. NeverMarried
5. Occupation_Govt_job
6. Occupation_Never_worked
7. Occupation_Private
8. Occupation_Self-employed
9. Occupation_children
10. LivesIn
11. GlucoseLevel
12. BMI
13. SmokingStatus_formerly smoked
14. SmokingStatus_never smoked
15. SmokingStatus_smokes

NOTE:
1. Make sure to preprocess the features in the above order exactly. Answer(s) of later question(s) depend(s) upon correct order of featuring processing.
2. You may have to use multiple instances of a trasnformer for this question.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import  SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

ord_pipe_with_imp = Pipeline([('impute', SimpleImputer(strategy="most_frequent")),
                     ("encoder", OrdinalEncoder())]) #gender, livesin



ord_pipe = Pipeline([("encoder", OrdinalEncoder())]) #HasTension, AnyHeartDisease


num_std_with_imp = Pipeline([('impute', SimpleImputer(strategy="most_frequent")),
                     ("std", StandardScaler())]) #Age, BMI

cat_ohe = Pipeline([('ohe', OneHotEncoder(handle_unknown="ignore"))]) #Occupation


num_mx_with_imp = Pipeline([('impute', SimpleImputer()),
                     ("std", MinMaxScaler())]) #GlucoseLevel


cat_ohe_with_imp = Pipeline([('impute', SimpleImputer(strategy="most_frequent")),
 ('ohe', OneHotEncoder(handle_unknown="ignore"))])



trf = ColumnTransformer([('ord_imp', ord_pipe_with_imp, ['Gender', "LivesIn"]),
                         ("ord", ord_pipe, ["HasTension", "AnyHeartDisease"]),
                         ('imp_std', num_std_with_imp, ["Age", "BMI"]),
                         ("ohe", cat_ohe, ["Occupation"]),
                         ("imp_mx", num_mx_with_imp, ["GlucoseLevel"]),
                         ('ohe_imp', cat_ohe_with_imp, ["SmokingStatus"])])

trf.fit(X_train)
X_train_trf = trf.transform(X_train)
X_test_trf = trf.transform(X_test)

In [ ]:
#.fit ----> learn the parameters(X_train(training))
#.transform -----> use the learned parameters(X_train), transform the incoming data

In [ ]:
trf

ColumnTransformer(transformers=[('ord_imp',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OrdinalEncoder())]),
                                 ['Gender', 'LivesIn']),
                                ('ord',
                                 Pipeline(steps=[('encoder',
                                                  OrdinalEncoder())]),
                                 ['HasTension', 'AnyHeartDisease']),
                                ('imp_std',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('std', StandardScaler())]),
                                 ['Age', 'BMI']),
                                ('ohe',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Occupation']),
                                ('imp_mx',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('std', MinMaxScaler())]),
                                 ['GlucoseLevel']),
                                ('ohe_imp',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['SmokingStatus'])])

In [ ]:
X_train_trf.shape

(2800, 16)

##Q3) Calculate the shape of the feature matrix of training dataset.

In [ ]:
X_train_trf.shape

(2800, 16)

##Q4) What is the mean of the transformed test data (features only)?

In [ ]:
import numpy as np
np.mean()

0.2467546478406633

 ## Q5) If you eliminate 1 feature with recursive feature elimination, which feature will be eliminated?

* Type the index of the eliminated feature (index starts from 0).

* Use LogisticRegression model with random state as 1729 and rest of the parameters with default values, as an estimator.

* Use processed training data.





In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

num_features_select = 15

rfe = RFE(LogisticRegression(random_state = 1729),
          n_features_to_select=num_features_select)
rfe.fit(X_train_trf, y_train)

RFE(estimator=LogisticRegression(random_state=1729), n_features_to_select=15)

In [ ]:
rfe.support_

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True])